# Setup

In [3]:
def clone_github_repo():
    !git clone https://github.com/Unique-Divine/SA-Project
    !mv "SA-Project" lib
    !ls
  
clone_github_repo()

Cloning into 'SA-Project'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 92 (delta 40), reused 65 (delta 21), pack-reused 0
Unpacking objects: 100% (92/92), done.
lib  sample_data


In [4]:
def setup_repo_imports():
    !pip install --quiet pytorch-lightning
    !pip install -q pytest==6.2
    !pip install -q tensorboard
    # !cp lib/__init__.py __init__.py
    import lib
    import os
    import sys
    path_to_lib = os.path.join("content", "lib")
    current_dir = os.getcwd()
    start_with_slash = lambda s: '/' + s if s[0] is not '/' else s
    path_to_lib, current_dir = [
        start_with_slash(s) for s in [path_to_lib, current_dir]]
    !cd lib && ls
    if not (path_to_lib == current_dir):
        os.chdir('lib')

def pass_tests():
    import lib.data_modules as data_modules
    import lib.lit_modules as lit_modules
    from lib import test_project

    test_project.TestMNISTOptimizers().test_quick_pass()

setup_repo_imports()
pass_tests()

data		 LICENSE	 models.py	  __pycache__  science.ipynb
data_modules.py  lightning_logs  optimization.py  README.md    test_project.py
__init__.py	 lit_modules.py  papers		  SA-Project


GPU available: True, used: False
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList       | 78.8 K
---------------------------------------------------
78.8 K    Trainable params
0         Non-trainable params
78.8 K    Total params
0.315     Total estimated model params size (MB)


# Optimizer comparisons with Multilayer Perceptrons 



In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from lib import test_project
from lib import data_modules
from lib import lit_modules
from typing import Generator
from pytorch_lightning.callbacks import early_stopping
early_stop_callback = early_stopping.EarlyStopping(
    monitor='val_loss',
    min_delta=0.00, 
    patience=3, 
    verbose=False, 
    mode='max'
)

optimizing_fns: dict = test_project.TestMNISTOptimizers().optimizing_fns()

def run_experiment(optimizing_fn, num_hidden_layers: int, gpus: int, 
                   progress_bar: bool = False, fast_dev_run: bool = False):
    progress_bar_refresh_rate = 0
    if progress_bar:
        progress_bar_refresh_rate = 50
    data_module = data_modules.MNISTDataModule()
    mnist_img_dims = (1, 28, 28)
    channels, width, height = mnist_img_dims
    network = lit_modules.LitFFNN(
        loss_fn=nn.CrossEntropyLoss(), 
        optimizing_fn=optimizing_fn, 
        mode='classifier', 
        num_hidden_layers=num_hidden_layers, 
        num_classes=10, 
        input_dim = channels * width * height
        )
    trainer = pl.Trainer(gpus=gpus, fast_dev_run=fast_dev_run,
                         progress_bar_refresh_rate=progress_bar_refresh_rate,
                         callbacks=[early_stop_callback], max_epochs=6)
    trainer.fit(model=network, datamodule=data_module)
    trainer.test(model=network, datamodule=data_module)


If you switch to a gpu runtime, this should only take 3-5 minutes to run. To use the GPU, simply switch 'GPUs' to 1. 

In [15]:
# Make sure things are running smoothly.

GPUs: int = 1

def experiment_settings() -> Generator:
    for num_hidden_layers in [0, 1, 2, 4, 8]:
        for optimizer_name in optimizing_fns.keys():
            yield num_hidden_layers, optimizer_name
            
for num_hidden_layers, optimizer_name in experiment_settings():   
    run_experiment(
        optimizing_fn=optimizing_fns[optimizer_name], 
        num_hidden_layers=num_hidden_layers, 
        gpus=GPUs, 
        progress_bar = False, 
        fast_dev_run = True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList       | 70.8 K
---------------------------------------------------
70.8 K    Trainable params
0         Non-trainable params
70.8 K    Total params
0.283     Total estimated model params size (MB)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.274984836578369, 'val_loss_epoch': 2.274984836578369}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.3180124759674072, 'val_loss_epoch': 2.3180124759674072}
--------------------------------------------------------------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList       | 70.8 K
---------------------------------------------------
70.8 K    Trainable params
0         Non-trainable params
70.8 K    Total params
0.283     Total estimated model params size (MB)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLo

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.3035459518432617, 'val_loss_epoch': 2.3035459518432617}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.305178642272949, 'val_loss_epoch': 2.305178642272949}
--------------------------------------------------------------------------------


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList       | 78.8 K
---------------------------------------------------
78.8 K    Trainable params
0         Non-trainable params
78.8 K    Total params
0.315     Total estimated model params size (MB)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList

...

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.30283260345459, 'val_loss_epoch': 2.30283260345459}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.3017683029174805, 'val_loss_epoch': 2.3017683029174805}
--------------------------------------------------------------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList       | 134 K 
---------------------------------------------------
134 K     Trainable params
0         Non-trainable params
134 K     Total params
0.539     Total estimated model params size (MB)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.3217694759368896, 'val_loss_epoch': 2.3217694759368896}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.287750244140625, 'val_loss_epoch': 2.287750244140625}
--------------------------------------------------------------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList       | 134 K 
---------------------------------------------------
134 K     Trainable params
0         Non-trainable params
134 K     Total params
0.539     Total estimated model params size (MB)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList       | 134 K 
---------------------------------------------------
134 K     Trainable params
0         Non-trainable params
134 K     Total params
0.539     Tota

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.2991201877593994, 'val_loss_epoch': 2.2991201877593994}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.30783748626709, 'val_loss_epoch': 2.30783748626709}
--------------------------------------------------------------------------------


In [16]:
for num_hidden_layers, optimizer_name in experiment_settings():   
    run_experiment(
        optimizing_fn=optimizing_fns[optimizer_name], 
        num_hidden_layers=num_hidden_layers, 
        gpus=GPUs, 
        progress_bar = True, 
        fast_dev_run = False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList       | 70.8 K
---------------------------------------------------
70.8 K    Trainable params
0         Non-trainable params
70.8 K    Total params
0.283     Total estimated model params size (MB)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList       | 70.8 K
---------------------------------------------------
70.8 K    Trainable params
0         Non-trainable params
70.8 K    Total params
0.283     Total estimated model params size (MB)



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 0.190912127494812, 'val_loss_epoch': 0.119410939514637}
--------------------------------------------------------------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_function | CrossEntropyLoss | 0     
1 | layers        | ModuleList       | 134 K 
---------------------------------------------------
134 K     Trainable params
0         Non-trainable params
134 K     Total params
0.539     Total estimated model params size (MB)



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.3073952198028564, 'val_loss_epoch': 2.309152364730835}
--------------------------------------------------------------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2.3095428943634033, 'val_loss_epoch': 2.3051059246063232}
--------------------------------------------------------------------------------


In [17]:
# To download the training logs for tensorboard
!zip -r logs.zip lightning_logs/ 

  adding: lightning_logs/ (stored 0%)
  adding: lightning_logs/version_0/ (stored 0%)
  adding: lightning_logs/version_0/hparams.yaml (stored 0%)
  adding: lightning_logs/version_0/events.out.tfevents.1619359165.d62941e2c864.57.1 (deflated 70%)
  adding: lightning_logs/version_0/events.out.tfevents.1619359028.d62941e2c864.57.0 (deflated 70%)
  adding: lightning_logs/version_0/checkpoints/ (stored 0%)
  adding: lightning_logs/version_0/checkpoints/epoch=5-step=3239.ckpt (deflated 12%)
  adding: lightning_logs/version_9/ (stored 0%)
  adding: lightning_logs/version_9/events.out.tfevents.1619359571.d62941e2c864.215.14 (deflated 33%)
  adding: lightning_logs/version_9/hparams.yaml (stored 0%)
  adding: lightning_logs/version_9/events.out.tfevents.1619359572.d62941e2c864.215.15 (deflated 70%)
  adding: lightning_logs/version_9/checkpoints/ (stored 0%)
  adding: lightning_logs/version_9/checkpoints/epoch=0-step=116.ckpt (deflated 9%)
  adding: lightning_logs/version_22/ (stored 0%)
  adding: